Detectron2 using cocolike structure training mask rcnn 50 layer.

data: module_preprocessing.py
data augmentations: https://jss367.github.io/Data-Augmentation-with-Detectron2.html
- flip horisontal 50% prob
- flip vertical 50% prob
- random rotation -20 to 20%. 
- random lightning 0.05 standard deviations. 

Iterations:
- 50% iterations as 256x256 images
- 25% iterations as 512x512 images
- 25% iterations as 756x756 images
- with variable optimized learning rate. 

Metrics: tensorboard (http://127.0.0.1:6006)

Submittion: module_submission.py

In [1]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available(), torch.version.cuda)

1.6.0 True 10.1


In [2]:
import os
import copy
import json
import pycocotools
import random 

import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
from collections import defaultdict

assert torch.__version__.startswith("1.6")

import detectron2
import detectron2.data.transforms as T
import detectron2.utils.comm as comm

from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg

from detectron2.data import MetadataCatalog,DatasetMapper,build_detection_train_loader,build_detection_test_loader
from detectron2.data import detection_utils as utils
from detectron2.data.catalog import DatasetCatalog
from detectron2.data.datasets import register_coco_instances 

from detectron2.evaluation import COCOEvaluator, inference_on_dataset

from detectron2.projects.deeplab import add_deeplab_config, build_lr_scheduler

In [3]:
PATH = os.path.abspath(os.getcwd())

register_coco_instances("my_dataset_train_v1",{},PATH + "/input/train_annotations.json",PATH + "/input/train_v2/")
register_coco_instances("my_dataset_val_v1",{},PATH + "/input/test_annotations.json",PATH + "/input/train_v2/")

my_dataset_train_metadata = MetadataCatalog.get("my_dataset_train_v1")
dataset_dicts = DatasetCatalog.get("my_dataset_train_v1")

[11/29 22:37:04 d2.data.datasets.coco]: Loaded 38301 images in COCO format from /application/input/train_annotations.json


In [4]:
def custom_mapper(dataset_dict):
    dataset_dict = copy.deepcopy(dataset_dict)  # it will be modified by code below
    image = utils.read_image(dataset_dict["file_name"], format="BGR")
    # List of transforms https://detectron2.readthedocs.io/modules/data_transforms.html
    # Add saturation, add shear orsmth.
    transform_list = [
                      T.RandomFlip(prob=0.5, horizontal=False, vertical=True),
                      T.RandomFlip(prob=0.5, horizontal=True, vertical=False),
                      T.RandomLighting(0.05),
                      T.RandomRotation((-0.2,0.2))
                     ]
    image, transforms = T.apply_transform_gens(transform_list, image)
    dataset_dict["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32"))

    annos = [
        utils.transform_instance_annotations(obj, transforms, image.shape[:2])
        for obj in dataset_dict.pop("annotations")
        if obj.get("iscrowd", 0) == 0
    ]
    instances = utils.annotations_to_instances(annos, image.shape[:2])
    dataset_dict["instances"] = utils.filter_empty_instances(instances)
    return dataset_dict

In [5]:
class CocoTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            os.makedirs("coco_eval", exist_ok=True)
            output_folder = "coco_eval"
        return COCOEvaluator(dataset_name, cfg, False, output_folder)
    
    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=custom_mapper)
    
    @classmethod
    def build_lr_scheduler(cls, cfg, optimizer):
        return build_lr_scheduler(cfg, optimizer)


In [10]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))

cfg.DATASETS.TRAIN = ("my_dataset_train_v1",) 
cfg.DATASETS.TEST = ("my_dataset_val_v1",)
cfg.TEST.EVAL_PERIOD = 5000
cfg.DATALOADER.NUM_WORKERS = 4 ## 4 per gpu
cfg.SOLVER.IMS_PER_BATCH = 8 
cfg.SOLVER.BASE_LR = 0.001  # pick a good LR
cfg.SOLVER.MAX_ITER = 40000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ship)
cfg.MAX_SIZE_TRAIN = 256 #Max image size 
#cfg.OUTPUT_DIR = "./run"

In [7]:
#https://stackoverflow.com/questions/48819139/tensorboard-link-wont-load-tensorflow-graphs-page-in-browser-the-connection-wa#51972959
#http://127.0.0.1:6006
%tensorboard --logdir=output --host=0.0.0.0
#%tensorboard --logdir=run --host=0.0.0.0

Reusing TensorBoard on port 6006 (pid 46), started 1 day, 4:07:57 ago. (Use '!kill 46' to kill it.)

Known TensorBoard instances:
  - port 6006: logdir output (started 1 day, 4:07:57 ago; pid 46)
Selecting TensorBoard with logdir output (started 1 day, 4:07:57 ago; port 6006, pid 46).


In [11]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg) 
trainer.resume_or_load(resume=True) #True takes last checkpoint file which is saved below.
trainer.train() #Trainer will throw out non-annotated pictures. 

[11/30 00:53:54 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

[11/30 00:53:56 d2.data.datasets.coco]: Loading /application/input/train_annotations.json takes 1.11 seconds.
[11/30 00:53:56 d2.data.datasets.coco]: Loaded 38301 images in COCO format from /application/input/train_annotations.json
[11/30 00:53:56 d2.data.build]: Removed 0 images with no usable annotations. 38301 images left.
[11/30 00:53:58 d2.data.build]: Using training sampler TrainingSampler
[11/30 00:53:58 d2.data.common]: Serializing 38301 elements to byte tensors and concatenating them all ...
[11/30 00:53:59 d2.data.common]: Serialized dataset takes 19.64 MiB
[11/30 00:53:59 d2.engine.train_loop]: Starting training from iteration 30000
[11/30 00:54:16 d2.utils.events]:  eta: 2:22:00  iter: 30019  total_loss: 0.5918  loss_cls: 0.08268  loss_box_reg: 0.2313  loss_mask: 0.1998  loss_rpn_cls: 0.008848  loss_rpn_loc: 0.03435  time: 0.8534  data_time: 0.0414  lr: 0.001  max_mem: 5977M
[11/30 00:54:33 d2.utils.events]:  eta: 2:21:35  iter: 30039  total_loss: 0.6166  loss_cls: 0.0722  

[11/30 01:02:58 d2.utils.events]:  eta: 2:15:33  iter: 30619  total_loss: 0.5228  loss_cls: 0.05987  loss_box_reg: 0.2167  loss_mask: 0.2056  loss_rpn_cls: 0.005221  loss_rpn_loc: 0.02977  time: 0.8677  data_time: 0.0209  lr: 0.001  max_mem: 5977M
[11/30 01:03:15 d2.utils.events]:  eta: 2:15:18  iter: 30639  total_loss: 0.5716  loss_cls: 0.06371  loss_box_reg: 0.2568  loss_mask: 0.2075  loss_rpn_cls: 0.006949  loss_rpn_loc: 0.03194  time: 0.8679  data_time: 0.0216  lr: 0.001  max_mem: 5977M
[11/30 01:03:33 d2.utils.events]:  eta: 2:15:00  iter: 30659  total_loss: 0.5959  loss_cls: 0.0755  loss_box_reg: 0.2425  loss_mask: 0.2347  loss_rpn_cls: 0.009087  loss_rpn_loc: 0.03453  time: 0.8678  data_time: 0.0221  lr: 0.001  max_mem: 5977M
[11/30 01:03:50 d2.utils.events]:  eta: 2:14:47  iter: 30679  total_loss: 0.5692  loss_cls: 0.07639  loss_box_reg: 0.2524  loss_mask: 0.2024  loss_rpn_cls: 0.00525  loss_rpn_loc: 0.04003  time: 0.8680  data_time: 0.0226  lr: 0.001  max_mem: 5977M
[11/30 01:

[11/30 01:12:15 d2.utils.events]:  eta: 2:06:42  iter: 31259  total_loss: 0.541  loss_cls: 0.05889  loss_box_reg: 0.2365  loss_mask: 0.2065  loss_rpn_cls: 0.006315  loss_rpn_loc: 0.03215  time: 0.8685  data_time: 0.0218  lr: 0.001  max_mem: 5977M
[11/30 01:12:32 d2.utils.events]:  eta: 2:06:26  iter: 31279  total_loss: 0.554  loss_cls: 0.06766  loss_box_reg: 0.2385  loss_mask: 0.2057  loss_rpn_cls: 0.009582  loss_rpn_loc: 0.02298  time: 0.8686  data_time: 0.0217  lr: 0.001  max_mem: 5977M
[11/30 01:12:50 d2.utils.events]:  eta: 2:06:08  iter: 31299  total_loss: 0.5697  loss_cls: 0.06392  loss_box_reg: 0.2274  loss_mask: 0.2173  loss_rpn_cls: 0.009407  loss_rpn_loc: 0.03694  time: 0.8685  data_time: 0.0217  lr: 0.001  max_mem: 5977M
[11/30 01:13:07 d2.utils.events]:  eta: 2:05:50  iter: 31319  total_loss: 0.5207  loss_cls: 0.05568  loss_box_reg: 0.2196  loss_mask: 0.2166  loss_rpn_cls: 0.009172  loss_rpn_loc: 0.03027  time: 0.8686  data_time: 0.0223  lr: 0.001  max_mem: 5977M
[11/30 01:

[11/30 01:21:31 d2.utils.events]:  eta: 1:57:21  iter: 31899  total_loss: 0.5861  loss_cls: 0.06794  loss_box_reg: 0.2633  loss_mask: 0.2137  loss_rpn_cls: 0.007305  loss_rpn_loc: 0.03507  time: 0.8685  data_time: 0.0222  lr: 0.001  max_mem: 5977M
[11/30 01:21:49 d2.utils.events]:  eta: 1:57:03  iter: 31919  total_loss: 0.5541  loss_cls: 0.05998  loss_box_reg: 0.217  loss_mask: 0.1957  loss_rpn_cls: 0.00637  loss_rpn_loc: 0.0338  time: 0.8685  data_time: 0.0220  lr: 0.001  max_mem: 5977M
[11/30 01:22:06 d2.utils.events]:  eta: 1:56:46  iter: 31939  total_loss: 0.5481  loss_cls: 0.06243  loss_box_reg: 0.2242  loss_mask: 0.2205  loss_rpn_cls: 0.005122  loss_rpn_loc: 0.02416  time: 0.8684  data_time: 0.0217  lr: 0.001  max_mem: 5977M
[11/30 01:22:23 d2.utils.events]:  eta: 1:56:28  iter: 31959  total_loss: 0.5821  loss_cls: 0.06329  loss_box_reg: 0.2586  loss_mask: 0.2235  loss_rpn_cls: 0.007757  loss_rpn_loc: 0.02489  time: 0.8684  data_time: 0.0216  lr: 0.001  max_mem: 5977M
[11/30 01:2

...

[11/30 03:05:50 d2.utils.events]:  eta: 0:17:19  iter: 38799  total_loss: 0.507  loss_cls: 0.06145  loss_box_reg: 0.225  loss_mask: 0.2025  loss_rpn_cls: 0.006325  loss_rpn_loc: 0.03108  time: 0.8671  data_time: 0.0208  lr: 0.001  max_mem: 5977M
[11/30 03:06:08 d2.utils.events]:  eta: 0:17:02  iter: 38819  total_loss: 0.5488  loss_cls: 0.06825  loss_box_reg: 0.2312  loss_mask: 0.2097  loss_rpn_cls: 0.006019  loss_rpn_loc: 0.03002  time: 0.8671  data_time: 0.0218  lr: 0.001  max_mem: 5977M
[11/30 03:06:25 d2.utils.events]:  eta: 0:16:45  iter: 38839  total_loss: 0.5245  loss_cls: 0.05795  loss_box_reg: 0.2071  loss_mask: 0.2022  loss_rpn_cls: 0.005207  loss_rpn_loc: 0.02197  time: 0.8671  data_time: 0.0222  lr: 0.001  max_mem: 5977M
[11/30 03:06:42 d2.utils.events]:  eta: 0:16:28  iter: 38859  total_loss: 0.4932  loss_cls: 0.05039  loss_box_reg: 0.2099  loss_mask: 0.198  loss_rpn_cls: 0.005414  loss_rpn_loc: 0.02524  time: 0.8671  data_time: 0.0221  lr: 0.001  max_mem: 5977M
[11/30 03:0

[11/30 03:15:06 d2.utils.events]:  eta: 0:08:05  iter: 39439  total_loss: 0.5452  loss_cls: 0.0563  loss_box_reg: 0.2246  loss_mask: 0.2236  loss_rpn_cls: 0.00622  loss_rpn_loc: 0.02503  time: 0.8671  data_time: 0.0219  lr: 0.001  max_mem: 5977M
[11/30 03:15:23 d2.utils.events]:  eta: 0:07:48  iter: 39459  total_loss: 0.545  loss_cls: 0.06665  loss_box_reg: 0.2257  loss_mask: 0.2228  loss_rpn_cls: 0.005467  loss_rpn_loc: 0.03138  time: 0.8671  data_time: 0.0225  lr: 0.001  max_mem: 5977M
[11/30 03:15:40 d2.utils.events]:  eta: 0:07:30  iter: 39479  total_loss: 0.4971  loss_cls: 0.04996  loss_box_reg: 0.2083  loss_mask: 0.1948  loss_rpn_cls: 0.00486  loss_rpn_loc: 0.02608  time: 0.8671  data_time: 0.0217  lr: 0.001  max_mem: 5977M
[11/30 03:15:58 d2.utils.events]:  eta: 0:07:13  iter: 39499  total_loss: 0.4761  loss_cls: 0.0517  loss_box_reg: 0.2009  loss_mask: 0.2023  loss_rpn_cls: 0.004161  loss_rpn_loc: 0.02575  time: 0.8671  data_time: 0.0226  lr: 0.001  max_mem: 5977M
[11/30 03:16:

[11/30 03:23:34 d2.evaluation.evaluator]: Inference done 253/4255. 0.0528 s / img. ETA=0:04:07
[11/30 03:23:39 d2.evaluation.evaluator]: Inference done 331/4255. 0.0529 s / img. ETA=0:04:05
[11/30 03:23:45 d2.evaluation.evaluator]: Inference done 412/4255. 0.0530 s / img. ETA=0:03:59
[11/30 03:23:50 d2.evaluation.evaluator]: Inference done 492/4255. 0.0530 s / img. ETA=0:03:55
[11/30 03:23:55 d2.evaluation.evaluator]: Inference done 575/4255. 0.0530 s / img. ETA=0:03:49
[11/30 03:24:00 d2.evaluation.evaluator]: Inference done 655/4255. 0.0531 s / img. ETA=0:03:44
[11/30 03:24:05 d2.evaluation.evaluator]: Inference done 737/4255. 0.0531 s / img. ETA=0:03:39
[11/30 03:24:10 d2.evaluation.evaluator]: Inference done 818/4255. 0.0531 s / img. ETA=0:03:33
[11/30 03:24:15 d2.evaluation.evaluator]: Inference done 898/4255. 0.0531 s / img. ETA=0:03:29
[11/30 03:24:20 d2.evaluation.evaluator]: Inference done 979/4255. 0.0531 s / img. ETA=0:03:24
[11/30 03:24:25 d2.evaluation.evaluator]: Inferenc

[11/30 03:27:48 d2.engine.defaults]: Evaluation results for my_dataset_val_v1 in csv format:
[11/30 03:27:48 d2.evaluation.testing]: copypaste: Task: bbox
[11/30 03:27:48 d2.evaluation.testing]: copypaste: AP,AP50,AP75,APs,APm,APl
[11/30 03:27:48 d2.evaluation.testing]: copypaste: 51.2060,83.7708,53.3286,37.0360,75.0636,78.7285
[11/30 03:27:48 d2.evaluation.testing]: copypaste: Task: segm
[11/30 03:27:48 d2.evaluation.testing]: copypaste: AP,AP50,AP75,APs,APm,APl
[11/30 03:27:48 d2.evaluation.testing]: copypaste: 41.5960,78.8597,41.3407,27.3754,63.5077,67.0297
[11/30 03:27:48 d2.utils.events]:  eta: 0:00:00  iter: 39999  total_loss: 0.557  loss_cls: 0.06058  loss_box_reg: 0.2287  loss_mask: 0.2218  loss_rpn_cls: 0.0052  loss_rpn_loc: 0.02921  time: 0.8671  data_time: 0.0219  lr: 0.001  max_mem: 5977M
[11/30 03:27:48 d2.engine.hooks]: Overall training speed: 9998 iterations in 2:24:29 (0.8672 s / it)
[11/30 03:27:48 d2.engine.hooks]: Total training time: 2:33:46 (0:09:17 on hooks)
[11/3

Loading and preparing results...
DONE (t=0.42s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *segm*
COCOeval_opt.evaluate() finished in 1.08 seconds.
Accumulating evaluation results...
COCOeval_opt.accumulate() finished in 0.09 seconds.
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.416
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.789
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.413
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.274
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.635
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.670
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.279
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.473
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.48

In [12]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))

cfg.DATASETS.TRAIN = ("my_dataset_train_v1",) 
cfg.DATASETS.TEST = ("my_dataset_val_v1",)
cfg.TEST.EVAL_PERIOD = 5000
cfg.DATALOADER.NUM_WORKERS = 4 ## 4 per gpu
cfg.SOLVER.IMS_PER_BATCH = 8
cfg.SOLVER.BASE_LR = 0.00033  
cfg.SOLVER.MAX_ITER = 45000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ship)
cfg.MAX_SIZE_TRAIN = 512 #Max image size 
cfg.LR_SCHEDULER_NAME = "WarmupCosineLR" #avoid getting stuck in local minima. 
#cfg.OUTPUT_DIR = "./run" 


In [13]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg) 
trainer.resume_or_load(resume=True) #True takes last checkpoint file which is saved below.
trainer.train() #Trainer will throw out non-annotated pictures. 

[11/30 03:32:19 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

[11/30 03:32:21 d2.data.datasets.coco]: Loaded 38301 images in COCO format from /application/input/train_annotations.json
[11/30 03:32:21 d2.data.build]: Removed 0 images with no usable annotations. 38301 images left.
[11/30 03:32:23 d2.data.build]: Using training sampler TrainingSampler
[11/30 03:32:23 d2.data.common]: Serializing 38301 elements to byte tensors and concatenating them all ...
[11/30 03:32:24 d2.data.common]: Serialized dataset takes 19.64 MiB
[11/30 03:32:24 d2.engine.train_loop]: Starting training from iteration 40000
[11/30 03:32:42 d2.utils.events]:  eta: 1:14:11  iter: 40019  total_loss: 0.4288  loss_cls: 0.04698  loss_box_reg: 0.1362  loss_mask: 0.208  loss_rpn_cls: 0.006261  loss_rpn_loc: 0.0357  time: 0.8938  data_time: 0.0426  lr: 0.001  max_mem: 5977M
[11/30 03:33:00 d2.utils.events]:  eta: 1:13:56  iter: 40039  total_loss: 0.402  loss_cls: 0.04202  loss_box_reg: 0.1273  loss_mask: 0.1925  loss_rpn_cls: 0.004635  loss_rpn_loc: 0.03183  time: 0.8938  data_time:

[11/30 03:41:49 d2.utils.events]:  eta: 1:06:12  iter: 40619  total_loss: 0.4142  loss_cls: 0.04759  loss_box_reg: 0.1218  loss_mask: 0.2069  loss_rpn_cls: 0.004769  loss_rpn_loc: 0.03013  time: 0.9094  data_time: 0.0214  lr: 0.001  max_mem: 5977M
[11/30 03:42:07 d2.utils.events]:  eta: 1:05:54  iter: 40639  total_loss: 0.3898  loss_cls: 0.0398  loss_box_reg: 0.1098  loss_mask: 0.1842  loss_rpn_cls: 0.006609  loss_rpn_loc: 0.02811  time: 0.9093  data_time: 0.0228  lr: 0.001  max_mem: 6011M
[11/30 03:42:25 d2.utils.events]:  eta: 1:05:35  iter: 40659  total_loss: 0.4327  loss_cls: 0.04752  loss_box_reg: 0.107  loss_mask: 0.2137  loss_rpn_cls: 0.00672  loss_rpn_loc: 0.02621  time: 0.9089  data_time: 0.0224  lr: 0.001  max_mem: 6011M
[11/30 03:42:43 d2.utils.events]:  eta: 1:05:17  iter: 40679  total_loss: 0.4121  loss_cls: 0.04127  loss_box_reg: 0.1334  loss_mask: 0.1923  loss_rpn_cls: 0.005248  loss_rpn_loc: 0.02565  time: 0.9089  data_time: 0.0223  lr: 0.001  max_mem: 6011M
[11/30 03:4

[11/30 03:51:29 d2.utils.events]:  eta: 0:56:24  iter: 41259  total_loss: 0.3959  loss_cls: 0.04233  loss_box_reg: 0.1158  loss_mask: 0.1891  loss_rpn_cls: 0.00538  loss_rpn_loc: 0.0282  time: 0.9076  data_time: 0.0216  lr: 0.001  max_mem: 6011M
[11/30 03:51:48 d2.utils.events]:  eta: 0:56:06  iter: 41279  total_loss: 0.3668  loss_cls: 0.0473  loss_box_reg: 0.1256  loss_mask: 0.1927  loss_rpn_cls: 0.004535  loss_rpn_loc: 0.02579  time: 0.9076  data_time: 0.0221  lr: 0.001  max_mem: 6011M
[11/30 03:52:06 d2.utils.events]:  eta: 0:55:48  iter: 41299  total_loss: 0.4203  loss_cls: 0.04787  loss_box_reg: 0.1263  loss_mask: 0.2049  loss_rpn_cls: 0.004646  loss_rpn_loc: 0.02605  time: 0.9077  data_time: 0.0223  lr: 0.001  max_mem: 6011M
[11/30 03:52:24 d2.utils.events]:  eta: 0:55:29  iter: 41319  total_loss: 0.4046  loss_cls: 0.04465  loss_box_reg: 0.107  loss_mask: 0.1981  loss_rpn_cls: 0.003999  loss_rpn_loc: 0.01991  time: 0.9078  data_time: 0.0216  lr: 0.001  max_mem: 6011M
[11/30 03:52

[11/30 04:01:15 d2.utils.events]:  eta: 0:46:55  iter: 41899  total_loss: 0.3797  loss_cls: 0.03871  loss_box_reg: 0.1103  loss_mask: 0.1888  loss_rpn_cls: 0.004583  loss_rpn_loc: 0.01952  time: 0.9098  data_time: 0.0217  lr: 0.001  max_mem: 6011M
[11/30 04:01:33 d2.utils.events]:  eta: 0:46:37  iter: 41919  total_loss: 0.3715  loss_cls: 0.03892  loss_box_reg: 0.1192  loss_mask: 0.1937  loss_rpn_cls: 0.004565  loss_rpn_loc: 0.019  time: 0.9098  data_time: 0.0220  lr: 0.001  max_mem: 6011M
[11/30 04:01:52 d2.utils.events]:  eta: 0:46:21  iter: 41939  total_loss: 0.4053  loss_cls: 0.0421  loss_box_reg: 0.1309  loss_mask: 0.1899  loss_rpn_cls: 0.007141  loss_rpn_loc: 0.02737  time: 0.9098  data_time: 0.0218  lr: 0.001  max_mem: 6011M
[11/30 04:02:10 d2.utils.events]:  eta: 0:46:02  iter: 41959  total_loss: 0.3638  loss_cls: 0.04063  loss_box_reg: 0.1157  loss_mask: 0.1789  loss_rpn_cls: 0.004108  loss_rpn_loc: 0.02324  time: 0.9097  data_time: 0.0225  lr: 0.001  max_mem: 6011M
[11/30 04:0

...

[11/30 04:30:23 d2.utils.events]:  eta: 0:17:50  iter: 43819  total_loss: 0.3609  loss_cls: 0.04639  loss_box_reg: 0.1244  loss_mask: 0.1737  loss_rpn_cls: 0.004531  loss_rpn_loc: 0.03025  time: 0.9096  data_time: 0.0219  lr: 0.001  max_mem: 6011M
[11/30 04:30:41 d2.utils.events]:  eta: 0:17:32  iter: 43839  total_loss: 0.3971  loss_cls: 0.04131  loss_box_reg: 0.1202  loss_mask: 0.1987  loss_rpn_cls: 0.004503  loss_rpn_loc: 0.02453  time: 0.9096  data_time: 0.0220  lr: 0.001  max_mem: 6011M
[11/30 04:31:00 d2.utils.events]:  eta: 0:17:14  iter: 43859  total_loss: 0.3989  loss_cls: 0.04324  loss_box_reg: 0.1152  loss_mask: 0.2025  loss_rpn_cls: 0.00457  loss_rpn_loc: 0.02347  time: 0.9096  data_time: 0.0216  lr: 0.001  max_mem: 6011M
[11/30 04:31:18 d2.utils.events]:  eta: 0:16:56  iter: 43879  total_loss: 0.3729  loss_cls: 0.04591  loss_box_reg: 0.1127  loss_mask: 0.1961  loss_rpn_cls: 0.005827  loss_rpn_loc: 0.01913  time: 0.9096  data_time: 0.0223  lr: 0.001  max_mem: 6011M
[11/30 04

[11/30 04:40:05 d2.utils.events]:  eta: 0:08:09  iter: 44459  total_loss: 0.3643  loss_cls: 0.03809  loss_box_reg: 0.113  loss_mask: 0.1737  loss_rpn_cls: 0.0053  loss_rpn_loc: 0.02292  time: 0.9094  data_time: 0.0218  lr: 0.001  max_mem: 6011M
[11/30 04:40:23 d2.utils.events]:  eta: 0:07:51  iter: 44479  total_loss: 0.4392  loss_cls: 0.04453  loss_box_reg: 0.1367  loss_mask: 0.2056  loss_rpn_cls: 0.00646  loss_rpn_loc: 0.03359  time: 0.9094  data_time: 0.0227  lr: 0.001  max_mem: 6011M
[11/30 04:40:41 d2.utils.events]:  eta: 0:07:32  iter: 44499  total_loss: 0.4247  loss_cls: 0.04792  loss_box_reg: 0.1215  loss_mask: 0.2071  loss_rpn_cls: 0.005526  loss_rpn_loc: 0.03137  time: 0.9094  data_time: 0.0218  lr: 0.001  max_mem: 6011M
[11/30 04:41:00 d2.utils.events]:  eta: 0:07:14  iter: 44519  total_loss: 0.3754  loss_cls: 0.03698  loss_box_reg: 0.121  loss_mask: 0.1921  loss_rpn_cls: 0.003785  loss_rpn_loc: 0.02622  time: 0.9094  data_time: 0.0220  lr: 0.001  max_mem: 6011M
[11/30 04:41:

[11/30 04:48:47 d2.evaluation.evaluator]: Inference done 429/4255. 0.0521 s / img. ETA=0:03:49
[11/30 04:48:52 d2.evaluation.evaluator]: Inference done 515/4255. 0.0521 s / img. ETA=0:03:43
[11/30 04:48:57 d2.evaluation.evaluator]: Inference done 599/4255. 0.0521 s / img. ETA=0:03:38
[11/30 04:49:02 d2.evaluation.evaluator]: Inference done 684/4255. 0.0521 s / img. ETA=0:03:32
[11/30 04:49:07 d2.evaluation.evaluator]: Inference done 770/4255. 0.0521 s / img. ETA=0:03:27
[11/30 04:49:12 d2.evaluation.evaluator]: Inference done 855/4255. 0.0521 s / img. ETA=0:03:22
[11/30 04:49:17 d2.evaluation.evaluator]: Inference done 940/4255. 0.0522 s / img. ETA=0:03:17
[11/30 04:49:22 d2.evaluation.evaluator]: Inference done 1025/4255. 0.0522 s / img. ETA=0:03:12
[11/30 04:49:27 d2.evaluation.evaluator]: Inference done 1109/4255. 0.0522 s / img. ETA=0:03:07
[11/30 04:49:32 d2.evaluation.evaluator]: Inference done 1195/4255. 0.0522 s / img. ETA=0:03:01
[11/30 04:49:37 d2.evaluation.evaluator]: Infer

[11/30 04:52:38 d2.engine.defaults]: Evaluation results for my_dataset_val_v1 in csv format:
[11/30 04:52:38 d2.evaluation.testing]: copypaste: Task: bbox
[11/30 04:52:38 d2.evaluation.testing]: copypaste: AP,AP50,AP75,APs,APm,APl
[11/30 04:52:38 d2.evaluation.testing]: copypaste: 53.7100,85.6005,56.8896,39.0897,78.3180,84.0815
[11/30 04:52:38 d2.evaluation.testing]: copypaste: Task: segm
[11/30 04:52:38 d2.evaluation.testing]: copypaste: AP,AP50,AP75,APs,APm,APl
[11/30 04:52:38 d2.evaluation.testing]: copypaste: 44.0096,80.8361,44.4122,29.2407,66.5157,76.1726
[11/30 04:52:38 d2.utils.events]:  eta: 0:00:00  iter: 44999  total_loss: 0.4443  loss_cls: 0.04685  loss_box_reg: 0.1303  loss_mask: 0.2049  loss_rpn_cls: 0.006155  loss_rpn_loc: 0.03104  time: 0.9090  data_time: 0.0224  lr: 0.001  max_mem: 6011M
[11/30 04:52:38 d2.engine.hooks]: Overall training speed: 4998 iterations in 1:15:43 (0.9090 s / it)
[11/30 04:52:38 d2.engine.hooks]: Total training time: 1:20:12 (0:04:28 on hooks)
[1

Loading and preparing results...
DONE (t=0.31s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *segm*
COCOeval_opt.evaluate() finished in 0.82 seconds.
Accumulating evaluation results...
COCOeval_opt.accumulate() finished in 0.07 seconds.
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.440
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.808
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.444
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.292
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.665
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.762
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.293
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.492
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.49

In [14]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))

cfg.DATASETS.TRAIN = ("my_dataset_train_v1",) 
cfg.DATASETS.TEST = ("my_dataset_val_v1",)
cfg.TEST.EVAL_PERIOD = 5000
cfg.DATALOADER.NUM_WORKERS = 4 ## 4 per gpu
cfg.SOLVER.IMS_PER_BATCH = 4 
cfg.SOLVER.BASE_LR = 0.0001  # pick a good LR
cfg.SOLVER.MAX_ITER = 47500
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ship)
cfg.MAX_SIZE_TRAIN = 756 #Max image size 
cfg.LR_SCHEDULER_NAME = "WarmupCosineLR"
#cfg.OUTPUT_DIR = "./run"

In [15]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg) 
trainer.resume_or_load(resume=True) #True takes last checkpoint file which is saved below.
trainer.train() #Trainer will throw out non-annotated pictures. 

[11/30 04:56:56 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

[11/30 04:56:57 d2.data.datasets.coco]: Loading /application/input/train_annotations.json takes 1.28 seconds.
[11/30 04:56:57 d2.data.datasets.coco]: Loaded 38301 images in COCO format from /application/input/train_annotations.json
[11/30 04:56:58 d2.data.build]: Removed 0 images with no usable annotations. 38301 images left.
[11/30 04:57:00 d2.data.build]: Using training sampler TrainingSampler
[11/30 04:57:00 d2.data.common]: Serializing 38301 elements to byte tensors and concatenating them all ...
[11/30 04:57:01 d2.data.common]: Serialized dataset takes 19.64 MiB
[11/30 04:57:01 d2.engine.train_loop]: Starting training from iteration 45000
[11/30 04:57:11 d2.utils.events]:  eta: 0:20:19  iter: 45019  total_loss: 0.2853  loss_cls: 0.02904  loss_box_reg: 0.05653  loss_mask: 0.1706  loss_rpn_cls: 0.003709  loss_rpn_loc: 0.02257  time: 0.4972  data_time: 0.0269  lr: 0.001  max_mem: 6011M
[11/30 04:57:21 d2.utils.events]:  eta: 0:20:15  iter: 45039  total_loss: 0.3408  loss_cls: 0.03215

[11/30 05:02:13 d2.utils.events]:  eta: 0:15:38  iter: 45619  total_loss: 0.3656  loss_cls: 0.02932  loss_box_reg: 0.06061  loss_mask: 0.2204  loss_rpn_cls: 0.00766  loss_rpn_loc: 0.03308  time: 0.5021  data_time: 0.0112  lr: 0.001  max_mem: 6011M
[11/30 05:02:24 d2.utils.events]:  eta: 0:15:29  iter: 45639  total_loss: 0.3679  loss_cls: 0.0371  loss_box_reg: 0.06842  loss_mask: 0.1933  loss_rpn_cls: 0.007669  loss_rpn_loc: 0.02713  time: 0.5025  data_time: 0.0114  lr: 0.001  max_mem: 6011M
[11/30 05:02:34 d2.utils.events]:  eta: 0:15:19  iter: 45659  total_loss: 0.3269  loss_cls: 0.02853  loss_box_reg: 0.06195  loss_mask: 0.1943  loss_rpn_cls: 0.005309  loss_rpn_loc: 0.02623  time: 0.5026  data_time: 0.0115  lr: 0.001  max_mem: 6011M
[11/30 05:02:44 d2.utils.events]:  eta: 0:15:09  iter: 45679  total_loss: 0.3245  loss_cls: 0.03702  loss_box_reg: 0.08932  loss_mask: 0.1917  loss_rpn_cls: 0.005572  loss_rpn_loc: 0.02447  time: 0.5028  data_time: 0.0113  lr: 0.001  max_mem: 6011M
[11/30

[11/30 05:07:35 d2.utils.events]:  eta: 0:10:19  iter: 46259  total_loss: 0.3424  loss_cls: 0.02839  loss_box_reg: 0.06943  loss_mask: 0.219  loss_rpn_cls: 0.003694  loss_rpn_loc: 0.02666  time: 0.5022  data_time: 0.0116  lr: 0.001  max_mem: 6011M
[11/30 05:07:45 d2.utils.events]:  eta: 0:10:09  iter: 46279  total_loss: 0.3291  loss_cls: 0.02342  loss_box_reg: 0.0547  loss_mask: 0.2128  loss_rpn_cls: 0.006446  loss_rpn_loc: 0.02492  time: 0.5022  data_time: 0.0115  lr: 0.001  max_mem: 6011M
[11/30 05:07:55 d2.utils.events]:  eta: 0:09:59  iter: 46299  total_loss: 0.3573  loss_cls: 0.02945  loss_box_reg: 0.06298  loss_mask: 0.2135  loss_rpn_cls: 0.004363  loss_rpn_loc: 0.02389  time: 0.5022  data_time: 0.0111  lr: 0.001  max_mem: 6011M
[11/30 05:08:05 d2.utils.events]:  eta: 0:09:49  iter: 46319  total_loss: 0.3117  loss_cls: 0.02732  loss_box_reg: 0.05528  loss_mask: 0.1807  loss_rpn_cls: 0.003471  loss_rpn_loc: 0.0212  time: 0.5022  data_time: 0.0118  lr: 0.001  max_mem: 6011M
[11/30 

[11/30 05:12:59 d2.utils.events]:  eta: 0:04:59  iter: 46899  total_loss: 0.3743  loss_cls: 0.03603  loss_box_reg: 0.06322  loss_mask: 0.2309  loss_rpn_cls: 0.007601  loss_rpn_loc: 0.03713  time: 0.5028  data_time: 0.0116  lr: 0.001  max_mem: 6011M
[11/30 05:13:09 d2.utils.events]:  eta: 0:04:49  iter: 46919  total_loss: 0.3731  loss_cls: 0.03253  loss_box_reg: 0.0675  loss_mask: 0.2227  loss_rpn_cls: 0.004382  loss_rpn_loc: 0.0265  time: 0.5028  data_time: 0.0109  lr: 0.001  max_mem: 6011M
[11/30 05:13:19 d2.utils.events]:  eta: 0:04:39  iter: 46939  total_loss: 0.3423  loss_cls: 0.03162  loss_box_reg: 0.06884  loss_mask: 0.2148  loss_rpn_cls: 0.005266  loss_rpn_loc: 0.01676  time: 0.5027  data_time: 0.0113  lr: 0.001  max_mem: 6011M
[11/30 05:13:28 d2.utils.events]:  eta: 0:04:29  iter: 46959  total_loss: 0.3729  loss_cls: 0.03401  loss_box_reg: 0.06478  loss_mask: 0.2522  loss_rpn_cls: 0.00446  loss_rpn_loc: 0.02982  time: 0.5026  data_time: 0.0116  lr: 0.001  max_mem: 6011M
[11/30 

[11/30 05:18:05 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[11/30 05:18:05 d2.data.common]: Serializing 4255 elements to byte tensors and concatenating them all ...
[11/30 05:18:05 d2.data.common]: Serialized dataset takes 2.15 MiB
WARNING [11/30 05:18:05 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass tasks in directly
[11/30 05:18:05 d2.evaluation.evaluator]: Start inference on 4255 images
[11/30 05:18:06 d2.evaluation.evaluator]: Inference done 11/4255. 0.0505 s / img. ETA=0:03:47
[11/30 05:18:11 d2.evaluation.evaluator]: Inference done 97/4255. 0.0516 s / img. ETA=0:04:00
[11/30 05:18:16 d2.evaluation.evaluator]: Inference done 183/4255. 0.0519 s / img. ETA=0:03:56
[11/30 05:18:21 d2.evaluation.evaluator]: Inference done 270/4255. 0.0519 s / img. ETA=0:03:50
[11/30 05:18:26 d2.evaluation.evaluator]

[11/30 05:22:15 d2.engine.defaults]: Evaluation results for my_dataset_val_v1 in csv format:
[11/30 05:22:15 d2.evaluation.testing]: copypaste: Task: bbox
[11/30 05:22:15 d2.evaluation.testing]: copypaste: AP,AP50,AP75,APs,APm,APl
[11/30 05:22:15 d2.evaluation.testing]: copypaste: 52.1606,83.7707,54.0239,37.2963,77.8169,82.4283
[11/30 05:22:15 d2.evaluation.testing]: copypaste: Task: segm
[11/30 05:22:15 d2.evaluation.testing]: copypaste: AP,AP50,AP75,APs,APm,APl
[11/30 05:22:15 d2.evaluation.testing]: copypaste: 42.6025,78.5757,42.9405,27.4828,66.1037,75.2890


In [ ]:
## save the training.

from detectron2.checkpoint import DetectionCheckpointer

checkpointer = DetectionCheckpointer(trainer.model, save_dir="./")
checkpointer.save("ship_ml")  # save to save_dir